<a href="https://colab.research.google.com/github/Dami442/Nlp-Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification #
import torch
import requests # for grabbing data from Yelp
from bs4 import BeautifulSoup # extract data from Yelp data
import re # regex

Instantiate Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Encode and Calculate Sentiment

In [ ]:
tokens = tokenizer.encode("It was ood, but it could have been better. Great!", return_tensors="pt")

In [ ]:
# Decode
tokenizer.decode(tokens[0])

'[CLS] it was ood, but it could have been better. great! [SEP]'

In [ ]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.6289, -1.2100,  1.0610,  1.7601,  0.7315]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
int(torch.argmax(result.logits)) + 1 # sentiment score

4

In [ ]:
result.logits

tensor([[ 4.7535,  2.0292, -0.5950, -2.9932, -2.4214]],
       grad_fn=<AddmmBackward0>)

Collect Reviews

In [ ]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2') # request data from url
soup = BeautifulSoup(r.text, 'html.parser') # get texts
regex = re.compile('.*comment.*') # pass regex to soup
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
results[0].text

"Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future."

### load revies

In [ ]:
reviews[0]

"Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future."

load Reviews into DataFrame and score

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df['review'].iloc[0]

"Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future."

In [ ]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

3

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

,review,sentiment
0,Seated without a booking on a super busy Satur...,5
1,Don't come here expecting legit Mexican food b...,3
2,The food was decent not great.. We had the gu...,2
3,Out of all the restaurants that I tried in Syd...,5
4,"Food was okay, guacamole was below average. Se...",2
5,The food and service here was really good. It...,5
6,Visiting from Texas and decided to give this r...,5
7,"Great atmosphere, attentive service, solid mar...",3
8,We came here on a Thursday night @ 5pm and by ...,4
9,The food is fresh and tasty. The scallop cevi...,4


In [ ]:
df['review'].iloc[2]

'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to check on us.. I will not go back. Suggest you find a Mexican restaurant that really wants your business..'